In [1]:
from selenium import webdriver
from time import sleep

In [2]:
#インスタのユーザーネームとパスワードを変数に格納
USERNAME = "zhizhimize" #username
PASSWORD = "kazuyuki1105" #パスワード入力

In [3]:
driver = webdriver.Chrome("C:\Test_Folder\chromedrive_win32\chromedriver")
target_url = "https://www.instagram.com/"
driver.get(target_url)
sleep(3)

In [4]:
error_flg = False

### インスタグラムのログイン

In [5]:
try:
    username_input = driver.find_element_by_xpath("//input[@aria-label='電話番号、ユーザーネーム、メールアドレス']")
    username_input.send_keys(USERNAME) #USERNAMEをキー入力
    sleep(1)
    
    password_input = driver.find_element_by_xpath("//input[@aria-label='パスワード']")
    password_input.send_keys(PASSWORD)
    sleep(1)
    
    login_button = driver.find_element_by_xpath("//button[@type = 'submit']")
    login_button.submit()
    sleep(1)
    
    
except Exception:
    error_flg = True


In [6]:
if error_flg is False:
    try:
        sleep(1)
        notnow_button = driver.find_element_by_xpath("//button[text() = '後で']")
        notnow_button.click()        
    
    except Exception:
        pass #例外が起きると何も処理をしないpass

### インスタで画面スクロールを行う

In [7]:
#インスタログイン後paulnicklenのユーザー画面に行く処理
target_username = "paulnicklen"
if error_flg is False:
    try:
        target_profile_url = target_url + target_username
        driver.get(target_profile_url)
        sleep(3)
    except Exception:
        print("検索時にエラーが発生しました。")
        error_flg = True

In [8]:
#ユーザの投稿数を取得する
if error_flg is False:
    try:
        post_count = driver.find_element_by_xpath("//span[text()='投稿']").text
        post_count = post_count.replace("投稿","").replace("件","")
        print("投稿件数：" + post_count)
        post_count = int(post_count)
        if post_count > 12:
            scroll_count = int(post_count/12) + 1
            try:
                for i in range(scroll_count):
                    #画面スクロールの処理
                    driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
                    sleep(2)
                    if i > 5:
                        break
            except Exception:
                print("画面スクロール中にエラーが発生しました。")
                error_flg = True
    except Exception:
        print("投稿数が取得できませんでした。")
        error_flg = True

投稿件数：2541


### seleniumで画像をダウンロードする方法（selenium + beautifulsoup）

In [9]:
from bs4 import BeautifulSoup

In [10]:
if error_flg is False:
    try:
        post_count = driver.find_element_by_xpath("//span[text()='投稿']").text
        post_count = post_count.replace("投稿","").replace("件","")
        print("投稿件数：" + post_count)
        post_count = int(post_count)
        if post_count > 12:
            scroll_count = int(post_count/12) + 1
            try:
                all_images = []
                for i in range(scroll_count):
                    #画面の読み込み all_iamges に格納
                    soup = BeautifulSoup(driver.page_source,"html.parser")
                    for image in soup.find_all("img"):
                        all_images.append(image)                      
                    #画面スクロールの処理
                    driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
                    sleep(2)
                    if i > 5:
                        break
                        
                all_images = list(dict.fromkeys(all_images))
                
                for index, image in enumerate(all_images):
                    print("画像番号：" + str(index))
                    print("image['src']：" + image["src"],end="\n\n")
            except Exception:
                print("画面スクロール中にエラーが発生しました。")
                error_flg = True
    except Exception:
        print("投稿数が取得できませんでした。")
        error_flg = True

投稿件数：2541
画像番号：0
image['src']：https://scontent-nrt1-1.cdninstagram.com/v/t51.2885-19/s150x150/93756524_253798018988532_1669202571023089664_n.jpg?_nc_ht=scontent-nrt1-1.cdninstagram.com&_nc_ohc=uCSMNEtuVGAAX_csV3H&tp=1&oh=4d6d076f3c602d4f9a3a44d408abadcd&oe=602DB6F9

画像番号：1
image['src']：https://scontent-nrt1-1.cdninstagram.com/v/t51.2885-15/s150x150/120036300_244428553654500_1929639767703625721_n.jpg?_nc_ht=scontent-nrt1-1.cdninstagram.com&_nc_ohc=a82kRi7iBYkAX8iO-8E&tp=1&oh=1e86edfdf38d206391ac730b08f93a03&oe=602BE432

画像番号：2
image['src']：https://instagram.fkix2-1.fna.fbcdn.net/v/t51.2885-15/e15/c0.280.720.720a/s150x150/123024330_178617967145561_9132370145877553883_n.jpg?_nc_ht=instagram.fkix2-1.fna.fbcdn.net&_nc_cat=105&_nc_ohc=rmqai-GNKkkAX-8rtBH&tp=1&oh=287a2fa36fa8c13186b9b2750f0b341a&oe=6004BF8D

画像番号：3
image['src']：https://scontent-nrt1-1.cdninstagram.com/v/t51.2885-15/e15/c29.113.667.667a/s150x150/118298313_592769494751409_409502747180012543_n.jpg?_nc_ht=scontent-nrt1-1.cdninsta

In [11]:
import requests
import re
import os
import shutil

path = r"C:\Test_Folder\Instagram\photo"

if error_flg is False:
    try:
        for index, image in enumerate(all_images):
            filename = "image_" + str(index) + ".jpg"
            image_path = os.path.join(path,filename)
            image_link = image["src"]

            url_ptn = re.compile(r"^(http|https)://")
            res = url_ptn.match(image_link)
            if res:
                response = requests.get(image_link,stream=True)

                #withステートメントでファイルをバイナリーデータ書き込む
                with open(image_path, "wb") as file:
                    shutil.copyfileobj(response.raw,file)
    except Exception as e:
        print(e)
        print(str(index)+"番目の画像のダウンロード・保存時にエラーが発生しました。")
        print("画像へのリンク：" + image_link)
                